<a href="https://colab.research.google.com/github/charlie9526/TaxiFare/blob/master/catboost_final_sumbission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 . Mounting

In [431]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks/FAIR_TAXI

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/FAIR_TAXI


# 2 . Imports

In [0]:
import pandas as pd
from datetime import datetime
import random
from google.colab import files
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)              
import numpy as np                 
from scipy.stats import norm   
import warnings
warnings.filterwarnings('ignore')   
%matplotlib inline                  
from math import sin, cos, sqrt, atan2, radians
import math

from sklearn.cluster import KMeans

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC,LinearSVC
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier



# 3 . Read files

In [0]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 4 . Functions

In [0]:
#elts write a lambda function
def get_duration(x):
  if x['pickup_time']==None or x['drop_time']==None:
    return None
  
  try :
    if int(x['pickup_time'].split("/")[1])>12 or int(x['drop_time'].split("/")[1])>12:
      gap = datetime.strptime(x['pickup_time'],"%m/%d/%Y %H:%M")-datetime.strptime(x['drop_time'],"%m/%d/%Y %H:%M")
    else:
      if int(x['pickup_time'].split("/")[1]) != int(x['drop_time'].split("/")[1]):
        gap = datetime.strptime(x['pickup_time'],"%m/%d/%Y %H:%M")-datetime.strptime(x['drop_time'],"%m/%d/%Y %H:%M")
      else:
        gap = datetime.strptime(x['pickup_time'],"%d/%m/%Y %H:%M")-datetime.strptime(x['drop_time'],"%d/%m/%Y %H:%M")
    # print (gap.total_seconds()/3600,x['pickup_time'] ,x['drop_time'])
  except :
    # print(int(x['pickup_time'].split("/")[0])>12 or int(x['drop_time'].split("/")[0])>12)
    return None
  # print(abs(gap.total_seconds()/3600), x['drop_time'],x['pickup_time'])
  disti =  abs(gap.total_seconds())
  if x['duration']>0:
    return x['duration']
  else:
    return disti

class MakeTripDurationFeature(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['time_gap'] = X.apply(lambda x:get_duration(x), axis=1)
        X = X.drop(labels=['pickup_time','drop_time'],axis=1)
        return X

# Change the label of binary values
class MakeTripDurationFeature(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        y = y.apply(lambda x:get_binary(x))
        # print (y);
        return self
    def transform(self,X):
        return X

# debugerr 
class debugerr(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        print (pd.DataFrame(X));
        return self
    def transform(self,X):
        return X

# lambda function for change label
def get_binary(x):
  if x==None:
    return 0
  if x == "correct":
    # print (x);
    return 1
  else:
    return 0

# set distance column
class set_distance(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X['distance'] = X.apply(lambda x:get_distance(x), axis=1)
      # print(X)
      # X = X.drop(labels=['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
      return X
# lambda column get distance
def get_distance(x):
  R = 6373.0
  lat1 = radians(float(x['drop_lat']))
  lon1 = radians(float(x['drop_lon']))
  lat2 = radians(float(x['pick_lat']))
  lon2 = radians(float(x['pick_lon']))

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c
  return distance

# remove columns
class remove_colums_unwanted(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X = X.drop(labels=['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
      return X


def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0

def get_hour_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  h =  datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").hour
  if (h>10 & h<14) | (h>15 & h<19) | (h>6 & h <9):
    return 1
  else:
    return 0

################### Here we have to add fare_per_hour column
################### write a lambda function

def get_fare_per_min(x):
  if x["fare"]>=0 and x["duration"]>0:
    minutes = (x["duration"]-x['additional_fare'])/60.0
    val = x['fare']/minutes
    return val
  if x["duration"]==0:
    return 0
  return None

########### This is for getting fare per distance 
########### Lambda function
def get_fare_per_distance(x):
  if x['distance']==0:
    return 0
  if x['meter_waiting_fare']>0:
    return float(x['fare']-x['meter_waiting_fare']-x['additional_fare'])/x['distance']
  else:
    return float(x['fare']-x['additional_fare'])/x['distance']

# datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
def get_time_type(df):
  pickup_slot=[]
  both=2
  school=3
  office=4
  free=1
  for index, row in df.iterrows():
    pickup_datetime=row['pickup_time']
    both_start="6:00"
    both_end="9:30"
    school_start="13:30"
    school_end="14:30"
    office_start="16:30"
    office_end="19:30"

    pickup_time=datetime.strptime(pickup_datetime.split(" ")[1],'%H:%M')
    both_start_time=datetime.strptime(both_start,'%H:%M')
    both_end_time=datetime.strptime(both_end,'%H:%M')
    school_start_time=datetime.strptime(school_start,'%H:%M')
    school_end_time=datetime.strptime(school_end,'%H:%M')
    office_start_time=datetime.strptime(office_start,'%H:%M')
    office_end_time=datetime.strptime(office_end,'%H:%M')
    
    if(pickup_time> both_start_time and pickup_time<both_end_time):
      pickup_slot.append(both)
      # print(pickup_time,both)
    elif(pickup_time> school_start_time and pickup_time<school_end_time):
      pickup_slot.append(school)
      # print(pickup_time,school)
    elif(pickup_time>office_start_time and pickup_time<office_end_time):
      pickup_slot.append(office)
      # print(pickup_time,office)
    else:
      pickup_slot.append(free)
      # print(pickup_time,free)  
  return pickup_slot

################fare rate of waiting
def get_waiting_rate(x):
  if math.isnan(x['meter_waiting']):
    return 0
  if x['meter_waiting']==0:
    return 0
  if  x['meter_waiting_fare']>=0:
    return (float(x['meter_waiting_fare'])/x['meter_waiting'])*60
  else:
    return 0

#################distance per min
def get_velocity(x):
  if math.isnan(x['duration']) or math.isnan(x['distance']):
    return 0
  if x['duration']==0:
    return 0
  return (float(x['distance'])/x['duration'])*60

def get_const_drives(x):
  if (x['fare']-x['additional_fare']-x['meter_waiting_fare'])<60:
    return 1
  else:
    return 0
##################ratio of fare from total
def get_fare_ratio(x):
  if (x['fare'])==0:
    return 0
  return float(x['additional_fare']+x['meter_waiting_fare'])/(x['fare'])

# 5 . PreProcess

### Some pre steps


In [435]:
train_df.drop_duplicates()
print (None)

None


### Create feature_df and label

In [0]:
trip_id  = test_df['tripid']

features_df = train_df.loc[:, train_df.columns != 'label']
label_df = train_df['label'].apply(encoding_label).values

features_df = features_df.loc[:, features_df.columns != 'tripid']
test_df = test_df.loc[:, test_df.columns != 'tripid']

### Fill nuls

In [0]:
features_df['fare'].fillna(0,inplace=True)
test_df['fare'].fillna(0,inplace=True)

features_df['additional_fare'].fillna(0,inplace=True)
test_df['additional_fare'].fillna(0,inplace=True)
################### ugly indexes
bads = [203757140,213112837,193653017,209365576,190167541]
train_df = train_df[~(train_df['tripid'].isin(bads))]


### New columns

In [0]:

features_df['distance'] = features_df.apply(lambda x:get_distance(x), axis=1)
test_df['distance'] = test_df.apply(lambda x:get_distance(x), axis=1)

features_df['fare_per_distance'] = features_df.apply(lambda x:get_fare_per_distance(x),axis=1 )
test_df['fare_per_distance'] = test_df.apply(lambda x:get_fare_per_distance(x),axis=1 )

features_df['fare_per_min'] = features_df.apply(lambda x:get_fare_per_min(x),axis=1 )
test_df['fare_per_min'] = test_df.apply(lambda x:get_fare_per_min(x),axis=1 )

features_df['pickup_hour'] = features_df.apply(lambda x : get_hour_value(x,'pickup_time'),axis=1)
test_df['pickup_hour'] = test_df.apply(lambda x : get_hour_value(x,'pickup_time'),axis=1)

features_df['velocity'] = features_df.apply(lambda x:get_velocity(x),axis=1 )
test_df['velocity'] = test_df.apply(lambda x:get_velocity(x),axis=1 )

kmeans = KMeans(n_clusters=3, random_state=0)
features_df['city'] = kmeans.fit(features_df[['pick_lon','pick_lat','fare']]).predict(features_df[['pick_lon','pick_lat','fare']])
test_df['city'] = kmeans.fit(test_df[['pick_lon','pick_lat','fare']]).predict(test_df[['pick_lon','pick_lat','fare']])

features_df['wr'] = features_df.apply(lambda x : get_waiting_rate(x),axis=1)
test_df['wr'] = test_df.apply(lambda x : get_waiting_rate(x),axis=1)


### Drop Columns

In [0]:
features_df = features_df.drop(labels=['pickup_time','drop_time','meter_waiting_till_pickup','pick_lat','drop_lat','pick_lon','drop_lon'],axis=1)
test_df = test_df.drop(labels=['pickup_time','drop_time','meter_waiting_till_pickup','pick_lat','drop_lat','pick_lon','drop_lon'],axis=1)

### Pipeline's pre prcessors

In [0]:
numeric_cols = features_df.columns[features_df.dtypes != "object"].values


In [0]:
cat_preprocessing_steps=Pipeline([
  ('cat_one_hot',OneHotEncoder())
])


numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='mean'))
])


preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols),  
        # ("categorical",cat_preprocessing_steps,cat_features)
    ],
    remainder = "drop"
)

# 6 . Model

In [0]:
pca = PCA()



# 7 . Cross validation and result

In [0]:
# # for x in np.arange(20, 100, 1):
# for x in range(0,1,1):
#   estimators = RandomForestClassifier(
#       # bootstrap= False,
#       # criterion="entropy",
#       # random_state = 4,
#       # max_depth=67,
#       # max_features= 'sqrt',
#       # min_samples_leaf= 4,
#       # min_samples_split=5,
#       # n_estimators = 500
#   )
#   xgb = XGBClassifier(
#       max_depth=19,
#       learning_rate=6,
#       n_estimators=x
#   )
#   full_pipeline = Pipeline([
#       ("preprocessor", preprocessor),
#       # ("debuffe",debugerr()),
#       ("pca",pca),
#       ("esti", estimators)
#   ])
#   kfold = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
#   cv_results = cross_val_score(full_pipeline, features_df, label_df, cv=kfold, scoring='f1')
#   print(cv_results)
#   print('%s: %f (%f) for rs = %f ' % ("RF", cv_results.mean()*100, cv_results.std(),x))
#   print("\n")


RF: 97.592728 (0.000779) with rs = 4 md=67

## **Catboost**


In [444]:
!pip install catboost
from catboost import CatBoostClassifier,Pool
import statistics

full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    # ("debuffe",debugerr()),
    # ("pca",pca),
    # ("esti", estimators)
])


x_train,x_test,y_train,y_test =  train_test_split( features_df, label_df, test_size=0.33, random_state=42)
full_pipeline.fit(x_train)
x_train = full_pipeline.transform(x_train)
x_test = full_pipeline.transform(x_test)
evaluation_data = full_pipeline.transform(test_df)


data_pool = Pool(
    data=x_train,
    label=y_train
)

model = CatBoostClassifier(
  # iterations=1500,
  random_seed = 42,
  # depth = 10,
  eval_metric='F1'
)
  
model.fit(
    data_pool,
    eval_set=(x_test, y_test),
    verbose = 800
)

eval_pool = Pool(
    data = x_test,
    label = y_test
)

eval_result = model.eval_metrics(
    data = eval_pool,
    metrics = ['F1']
)


for metric_key in eval_result:
  print ("%s -: %f" % (metric_key,statistics.mean(eval_result[metric_key])))

predicted = model.predict(evaluation_data,prediction_type='Class')

data_dict = { 'tripid':trip_id, 'prediction':predicted}
pd_test_result = pd.DataFrame(data=data_dict,columns=['tripid','prediction'])
pd_test_result = pd_test_result.set_index('tripid') 
pd_test_result.to_csv("catboost_result.csv", index=True)

Learning rate set to 0.057938
0:	learn: 0.9633744	test: 0.9641427	best: 0.9641427 (0)	total: 12.3ms	remaining: 12.3s
800:	learn: 0.9914342	test: 0.9767352	best: 0.9773056 (678)	total: 9.06s	remaining: 2.25s
999:	learn: 0.9928643	test: 0.9762502	best: 0.9773056 (678)	total: 11.3s	remaining: 0us

bestTest = 0.9773056494
bestIteration = 678

Shrink model to first 679 iterations.
F1 -: 0.976497


F1 -: 0.976497

# 8 . Train model

In [0]:
# full_pipeline.fit(features_df,label_df)
# print (None)

# 9 . Predict and save result

In [0]:
# predicted = full_pipeline.predict(test_df)

# data_dict = { 'tripid':trip_id, 'prediction':predicted}
# pd_test_result = pd.DataFrame(data=data_dict,columns=['tripid','prediction'])
# pd_test_result = pd_test_result.set_index('tripid') 
# pd_test_result.to_csv("submission_result.csv", index=True)